# 从命令行配置超参数
在运行深度学习实验的时候，建议遵循一些良好的做法：
- 将配置和源码分开
- 保证实验的重复性
使用CLI（command line interface)可以帮助达到这个目的。

## 实现CLI
实现CLI只需要实例化`LightningCLI`类，以`LightningModule`和`LightningDataModule`(可选)作为参数。

In [ ]:
# main.py
from pytorch_lightning.cli import LightningCLI

# simple demo classes for your convenience
from pytorch_lightning.demos.boring_classes import DemoModel, BoringDataModule


def cli_main():
    cli = LightningCLI(DemoModel, BoringDataModule)
    # note: don't call fit!!


if __name__ == "__main__":
    cli_main()
    # note: it is good practice to implement the CLI in a function and call it in the main if block

现在便可以使用CLI控制模型了，可以通过以下命令查看可以使用的命令：
```shell
python main.py --help
```

可以添加的选项包括`LightningModule`和`LightningDataModule`的构造函数`__init__`的参数。

## 使用多模型和多数据集

```shell
# Mix and match anything
$ python main.py fit --model=GAN --data=MNIST
$ python main.py fit --model=Transformer --data=MNIST
```

### 多模型
在实例化`LightningCLI`时忽略`model_class`参数，便可以支持多模型。

In [ ]:
# main.py
from pytorch_lightning.cli import LightningCLI
from pytorch_lightning.demos.boring_classes import DemoModel


class Model1(DemoModel):
    def configure_optimizers(self):
        print("⚡", "using Model1", "⚡")
        return super().configure_optimizers()


class Model2(DemoModel):
    def configure_optimizers(self):
        print("⚡", "using Model2", "⚡")
        return super().configure_optimizers()


cli = LightningCLI(datamodule_class=BoringDataModule)

这样便可以从CLI选择模型
```shell
# use Model1
python main.py fit --model Model1

# use Model2
python main.py fit --model Model2
```



也可以给`model_class`传入一个基类，并且指定`subclass_mode_model=True`，而不是省略`model_class`参数，这将使程序只接受此基类下的子类。


### 多数据模块
和多模型类似，在实例化`LightningCLI`的时候忽略`datamodule_class`参数，可以让程序支持多数据模块。

In [ ]:
# main.py
import torch
from pytorch_lightning.cli import LightningCLI
from pytorch_lightning.demos.boring_classes import BoringDataModule


class FakeDataset1(BoringDataModule):
    def train_dataloader(self):
        print("⚡", "using FakeDataset1", "⚡")
        return torch.utils.data.DataLoader(self.random_train)


class FakeDataset2(BoringDataModule):
    def train_dataloader(self):
        print("⚡", "using FakeDataset2", "⚡")
        return torch.utils.data.DataLoader(self.random_train)


cli = LightningCLI(DemoModel)

```
# use Model1
python main.py fit --data FakeDataset1

# use Model2
python main.py fit --data FakeDataset2
```
类似的，给`datamodule_class`传入一个基类，并设置`subclass_mode_data=True`，可以让程序只接受此基类的子类。

### 多优化器
`torch.optim`当中的优化器开箱即用。
```
python main.py fit --optimizer AdamW
```
如果想要使用的优化器需要其它的参数，可以通过CLI添加，不需要改变代码！
```
python main.py fit --optimizer SGD --optimizer.lr=0.01
```
另外，任何基于`torch.optim.Optimizer`的定制优化器都可以用。

### 多schedulers
`torch.optim.lr_scheduler`中的学习率schedulers开箱即用：
```
python main.py fit --lr_scheduler CosineAnnealingLR
```
如果使用的scheduler需要其它参数，通过CLI添加，不需要改变代码！
```
python main.py fit --lr_scheduler=ReduceLROnPlateau --lr_scheduler.monitor=epoch
```
另外，任何基于`torch.optim.lr_scheduler.LRScheduler`的定制scheduler都可以使用。

### 使用其它包中的类
之前的例子中，可选择的类都在一个python文件中定义，另外在导入其它包中的类后，也可以直接使用

In [ ]:
from pytorch_lightning.cli import LightningCLI
import my_code.models  # noqa: F401
import my_code.data_modules  # noqa: F401
import my_code.optimizers  # noqa: F401

cli = LightningCLI()
# noqa：F401是为了避免pylint报错

也可以使用完整的导入路径。
```
python main.py fit --model my_code.models.Model1
```

当处于多模型模式，主帮助文档不包含这些模型或数据模块的参数，需要通过以下方式显示：
```
python main.py fit --model.help Model1
python main.py fit --data.help FakeDataset2
python main.py fit --optimizer.help Adagrad
python main.py fit --lr_scheduler.help StepLR
```

## YAML
随着项目变得越来越复杂，需要配置的参数变得非常多，让使用CLI变得不方便。为了解决这个问题，`LightningCLI`支持从配置文件接受输入。默认的配置文件格式为YAML。

使用yaml配置文件
```
python main.py fit --config config.yaml
```
单独给的参数可以覆盖配置文件中的参数
```
python main.py fit --config config.yaml --trainer.max_epochs 100
```

## 自动保存配置
`LightningCLI`会自动保存YAML配置在日志文件夹。以后可以通过这些保存的文件重复实验。

可以通过设置`save_config_callback=None`关闭自动保存。

In [ ]:
# 定义日志名
cli = LightningCLI(..., save_config_kwargs={"config_filename": "name.yaml"})

## 为CLI准备配置文件
从头开始编写配置可以既耗时又容易出错。为了缓解这种情况，CLI有`--print_config`参数，它可以打印配置到标准输出。
```
python main.py fit --print_config
```
如果使用多个模型，没有指定模型的情况下不会给出模型的超参数，所以可以指定模型后打印：
```
python main.py fit --model DemoModel --print_config
```

一个标准的实验流程可以是：
```
# Print a configuration to have as reference
python main.py fit --print_config > config.yaml
# Modify the config to your liking - you can remove all default arguments
nano config.yaml
# Fit your model using the edited configuration
python main.py fit --config config.yaml
```

另外可以使用多个配置文件，重复的项后面覆盖前面：
```
$ python main.py fit --config config_1.yaml --config config_2.yaml
```
可以分组传入参数：
```
$ python main.py fit --trainer trainer.yaml --model model.yaml --data data.yaml [...]
```

# 高级用法
根据子命令选择配置
```
# config.yaml
fit:
    trainer:
        max_steps: 100
test:
    trainer:
        max_epochs: 10
```
将配置文件放在子命令前：
```
# full routine with max_steps = 100
$ python main.py --config config.yaml fit

# test only with max_epochs = 10
$ python main.py --config config.yaml test
```

## 设置默认的配置文件

In [ ]:
cli = LightningCLI(MyModel, MyDataModule, parser_kwargs={"default_config_files": ["my_cli_defaults.yaml"]})
# or
cli = LightningCLI(MyModel, MyDataModule, parser_kwargs={"fit": {"default_config_files": ["my_fit_defaults.yaml"]}})

默认情况下CLI会自动运行输入的子命令，可以关闭自动运行，并手动运行

In [ ]:
cli = LightningCLI(MyModel, run=False)  # True by default
# you'll have to call fit yourself:
cli.trainer.fit(cli.model)

## 从python中运行
给`cli_main()`函数传入args参数。

In [ ]:
from pytorch_lightning.cli import ArgsType, LightningCLI


def cli_main(args: ArgsType = None):
    cli = LightningCLI(MyModel, ..., args=args)
    ...


if __name__ == "__main__":
    cli_main()

然后在另一个文件中导入并调用`cli_main()`，args可以传入一个字符串，字典或者 jsonargparse.Namespace

In [ ]:
args = {
    "trainer": {
        "max_epochs": 100,
    },
    "model": {},
}

args["model"]["encoder_layers"] = 8
cli_main(args)
args["model"]["encoder_layers"] = 12
cli_main(args)
args["trainer"]["max_epochs"] = 200
cli_main(args)
